In [38]:
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
import numpy as np

In [39]:

df = pd.read_csv('https://gist.githubusercontent.com/sachinsdate/09cfd42b7701c48ec68b04c786786434/raw/4b50e718a83a4b20adcff5552ccea8f1054ce919/fish.csv')

In [40]:
df.head()

,LIVE_BAIT,CAMPER,PERSONS,CHILDREN,FISH_COUNT
0,0,0,1,0,0
1,1,1,1,0,0
2,1,0,1,0,0
3,1,1,2,1,0
4,1,0,1,0,1


In [41]:
px.bar(df['FISH_COUNT'].value_counts())

Neste gráfico podemos ver a distribuição dos valores da variável FISH_COUNT, sendo possível observar a disparidade da ocorrência do valor 0 em relação aos demais valores. Tal distribuição é claramente uma distribuição de Poisson.

In [42]:
df.query('FISH_COUNT == 0').shape[0]/df.shape[0]

0.568

Confirmamos a alta recorrência do valor 0 na variável ao perceber que ela ocupa 56,8% da quantidade de entradas na variável.

In [43]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=df.index, y=df.FISH_COUNT),row=1, col=1)
fig.add_trace(go.Scatter(x=df.query('FISH_COUNT != 0').index, y=df.query('FISH_COUNT != 0').FISH_COUNT),row=1, col=2)


Neste momento realizamos uma comparação entre gráficos. A esquerda temos um gráfico de linha com o dataframe sem tratamento, onde é possível observar a predominância da linha em 0 no eixo y. Do lado direito temos o dataframe sem as entradas com valor 0, apresentando um gráfico mais ruidoso e mais interessante a nossa análise.

In [56]:
#define o ponto de corte do dataset de treino

def create_train_test(df):
    
    cut = 0.8
    index_cut = int(len(df) * cut)
    df_train= df.iloc[:index_cut,:]
    df_test = df.iloc[index_cut:,:]
    return (df_train, df_test)


df_train, df_test  = create_train_test(df)


Decidimos acima por deixar 20% do dataframe para validação e 80% para treinamento.

In [46]:
from patsy import dmatrices

expr = 'FISH_COUNT ~ LIVE_BAIT  + CAMPER + CHILDREN + PERSONS'

y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')

y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')



Importamos a função dmatrices para criar uma expressão padrão onde elencamos FISH_COUNT como variável dependente e LIVE_BAIT, CAMPER, CHILDREN e PERSONS, como variáveis independentes.

In [47]:
zip_training_results = sm.ZeroInflatedPoisson(endog=y_train, 
                                              exog=X_train, 
                                              exog_infl=X_train,
                                                inflation='logit').fit()

print(zip_training_results.summary())

         Current function value: 3.197480
         Iterations: 35
         Function evaluations: 37
         Gradient evaluations: 37
                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:              FISH_COUNT   No. Observations:                  200
Model:             ZeroInflatedPoisson   Df Residuals:                      195
Method:                            MLE   Df Model:                            4
Date:                 Thu, 08 Jun 2023   Pseudo R-squ.:                  0.3484
Time:                         16:56:51   Log-Likelihood:                -639.50
converged:                       False   LL-Null:                       -981.37
Covariance Type:             nonrobust   LLR p-value:                1.146e-146
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
inflate_Intercept     0.7800      1.297      0.601    

/home/furius/miniconda3/envs/myenv/lib/python3.11/site-packages/statsmodels/base/model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/home/furius/miniconda3/envs/myenv/lib/python3.11/site-packages/statsmodels/base/model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



Já conscientes de que a distribuição existente no dataset é uma distribuição de Poisson, usamos ZeroInflatedPoisson para treinar nosso modelo e obtivemos o output acima. Importante notar que as variáveis inflate_INTERCEPT, inflate_LIVE_BAIT e inflate_CAMPER tem um valor de P>[z] não nulo, tornando as estatisticamente não significantes. 

In [53]:

#zero inflated poisson
#y_train = df_train['FISH_COUNT']
#X_train = df_train.drop('FISH_COUNT', axis=1)
zip_training_results = sm.ZeroInflatedPoisson(endog=y_train, 
                                              exog=X_train, 
                                              exog_infl=X_train.drop(['Intercept', 'LIVE_BAIT','CAMPER'], axis=1),
                                                inflation='logit').fit()


print(zip_training_results.summary())


Optimization terminated successfully.
         Current function value: 3.214268
         Iterations: 21
         Function evaluations: 23
         Gradient evaluations: 23
                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:              FISH_COUNT   No. Observations:                  200
Model:             ZeroInflatedPoisson   Df Residuals:                      195
Method:                            MLE   Df Model:                            4
Date:                 Thu, 08 Jun 2023   Pseudo R-squ.:                  0.3449
Time:                         16:57:15   Log-Likelihood:                -642.85
converged:                        True   LL-Null:                       -981.37
Covariance Type:             nonrobust   LLR p-value:                3.258e-145
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
inflate_CHILDREN  

/home/furius/miniconda3/envs/myenv/lib/python3.11/site-packages/statsmodels/base/model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



Considerando o teste anterior decidimos por excluir as variáveis estatisticamente não significantes para tornar nosso modelo mais confiável e então o retreinamos.

In [50]:
zip_predictions = zip_training_results.predict(X_test,exog_infl=X_test.drop(['Intercept', 'LIVE_BAIT','CAMPER'], axis=1))

Realizamos as predições em cima dos dados de validação, mais uma vez tomando cuidado para não incluir as variáveis não significantes.

In [51]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_test.index, y=df_test.FISH_COUNT, name='obs'))
fig.add_trace(go.Scatter(x=zip_predictions.index, y=zip_predictions, name='pred'))

In [57]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error


print(mean_absolute_error(y_test, zip_predictions))
print(mean_squared_error(y_test, zip_predictions))
print(np.sqrt(mean_squared_error(y_test, zip_predictions)))
print(mean_absolute_percentage_error(y_test, zip_predictions))

1.625907206020679
7.218876134112521
2.686796630583067
2836587844899771.5


De acordo com as métricas acima o erro médio absoluto, o erro médio quadráticos e a raiz dos erros médios quadráticos são relativamente baixos, porém o erro percentual absoluto médio é bem alto, o que se deve a presença de outliers no dataset.